In [254]:
import zkbp

import json
from wurlitzer import pipes

values =32
BITS=32

gh = zkbp.gen_GH()
r = zkbp.gen_r()
cm = zkbp.commit(values,r, gh)
print(cm.x)
print(cm.y)
print((cm.sum(cm)).x)
print((cm.sum(cm)).y)
cm_cm = cm.sum(cm)

print(cm.get)
print(len(cm.get))

range_proof = None
range_proof_json = None
ret = None
with pipes() as (out,err):
    range_proof = zkbp.range_proof_single(n_bit=BITS, val=values, gh=gh, r=r)
    range_proof_json = json.loads(range_proof)
    # print(range_proof)
    # assert (zkbp.range_proof_single_verify(range_proof,BITS,gh,cm)) == True
    ret = json.loads(zkbp.range_proof_single_verify(range_proof,BITS,gh,cm))

def ptsol(point_str):
    return (int(zkbp.from_str(point_str["point"]).x()), int(zkbp.from_str(point_str["point"]).y()))

scalar_cmp = (int(ret[0]["scalar"],16))
point_cmp = (ptsol(ret[1]))
print(ret[2])
z_cmp = (int(ret[2],16))

import hashlib
kzgenlabel = 1264215406
# print(zkbp.ghvec(BITS))
ghvec =json.loads(zkbp.ghvec(BITS))
# print(len(ghvec[0]))


A = range_proof_json["A"]
print("123123",zkbp.from_str(A["point"]).x() )
A = (int(zkbp.from_str(A["point"]).x()),int(zkbp.from_str(A["point"]).y()))
print(A)
S = range_proof_json["S"]
S = (int(zkbp.from_str(S["point"]).x()),int(zkbp.from_str(S["point"]).y()))


A_bytes = zkbp.from_str(range_proof_json["A"]["point"]).to_bytes() 
print(A_bytes)
stdout = (out.read())
inner_cmp = ptsol(ret[-1][0])
print(inner_cmp)

<built-in method x of builtins.Commit object at 0x105c7bc90>
<built-in method y of builtins.Commit object at 0x105c7bc90>
<built-in method x of builtins.Commit object at 0x105c79bc0>
<built-in method y of builtins.Commit object at 0x105c79bc0>
7c21fdcc7e6a055a6995bc27f5343700e80e61ef8aba2890d96a6ac3500c4723
64
02d974f6b43e657914a7bdf36831a4505ba41d25e748e361bd65e9839e59c21c
123123 288045434081726416572067367057073206317055333641009491015611614907605514551
(288045434081726416572067367057073206317055333641009491015611614907605514551, 20772281073872014750388646511477327178152376773644613294912510856912178788899)
[55, 249, 228, 55, 20, 111, 96, 10, 0, 152, 139, 11, 30, 88, 226, 234, 37, 126, 224, 224, 230, 104, 43, 20, 202, 8, 46, 245, 38, 7, 163, 0, 35, 58, 120, 18, 167, 186, 100, 106, 252, 220, 154, 87, 143, 214, 51, 34, 58, 235, 94, 111, 76, 124, 35, 139, 23, 115, 234, 58, 199, 177, 236, 45]
(4092044273462250100196902779019191890337916830103873700894858447423788306484, 18219337238173861

In [255]:
from solcx import compile_source, compile_standard, install_solc, import_installed_solc, link_code
from web3 import Web3 
from ecpy.curves     import Curve,Point

http_connection = "HTTP://127.0.0.1:8545"
w3 = Web3(Web3.HTTPProvider(http_connection,request_kwargs={"timeout":600}))
chain_id = 1337

secret_key = "0x8f2a55949038a9610f50fb23b5883af3b4ecb3c3bb792cbcefbd1542c692be63"
public_key= "0209b02f8a5fddd222ade4ea4528faefc399623af3f736be3c44f03e2df22fb792"
sk = int(secret_key,16)
cv = Curve.get_curve('secp256k1')
# order = cv.order
G = Point(0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798, 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8, curve=cv)
pk = sk * G
print(pk)
eth_account = w3.eth.account.from_key(sk)
print(eth_account.address)

# After 0.8.20 the opcodes have been changed.
install_solc("0.8.19")

print(int("0x2523648240000001BA344D80000000086121000000000013A700000000000013",16))
order = int("0x2523648240000001BA344D8000000007FF9F800000000010A10000000000000D",16) #website
order = 21888242871839275222246405745257275088548364400416034343698204186575808495617 #ark
basefield =21888242871839275222246405745257275088696311157297823662689037894645226208583 # ark #https://hackmd.io/@jpw/bn254
print(order)
print("21888242871839275222246405745257275088548364400416034343698204186575808495617")

(0x9b02f8a5fddd222ade4ea4528faefc399623af3f736be3c44f03e2df22fb792 , 0xf3931a4d9573d333ca74343305762a753388c3422a86d98b713fc91c1ea04842)
0xFE3B557E8Fb62b89F4916B721be55cEb828dBd73
16798108731015832284940804142231733909889187121439069848933715426072753864723
21888242871839275222246405745257275088548364400416034343698204186575808495617
21888242871839275222246405745257275088548364400416034343698204186575808495617


In [256]:
import random
import functools
import os

file_name = "testchain.sol"
cont_name= "Bulletproof"
def get_contract(file_name):
    with open(file_name) as f:
        return f.read()
    
contract_source_code = get_contract(file_name)

library_file = "bn254.sol"
library_name = "libBN254"
library_source_code = get_contract(library_file)


def compile(contract_source_code, file_name):
# Standard JSON input for compilation
    standard_input = {
        "language": "Solidity",
        "sources": {
            file_name: {
                "content": contract_source_code
            }
        },
        "settings": {
            "optimizer":{
                "enabled": True
            },
            "viaIR": True,
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]
                }
            }
        }
    }

    # Compile the contract
    compiled_output = compile_standard(
        standard_input,
        allow_paths=".",
        solc_version="0.8.19"
    )
    return compiled_output

# library_compiled_output = compile(library_source_code, library_file)
# abi = library_compiled_output["contracts"][library_file][library_name]["abi"]
# bytecode = library_compiled_output["contracts"][library_file][library_name]["evm"]["bytecode"]["object"]
# library_contract = w3.eth.contract(abi =abi, bytecode=bytecode)

In [257]:
def deploy_contract(w3,sk,contract):
    transaction = None
    try:
        account_address = Web3.toChecksumAddress(eth_account.address)
        nonce = w3.eth.get_transaction_count(account_address)
        transaction = contract.constructor().buildTransaction(
            {"chainId":chain_id, "from":account_address, "nonce":nonce}
        )
    except Exception as e:
        print(e)
        assert False, "Unable to init transaction"

    signed_tx = w3.eth.account.sign_transaction(transaction,private_key=sk)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    return tx_receipt.contractAddress

# lib_contract_address = deploy_contract(w3,sk, library_contract)
# lib_contract_isntance = w3.eth.contract(lib_contract_address, abi=abi)

In [258]:
compiled_output = compile(contract_source_code, file_name)
print(compiled_output)
abi = compiled_output["contracts"][file_name][cont_name]["abi"]
bytecode = compiled_output["contracts"][file_name][cont_name]["evm"]["bytecode"]["object"]
# linked_bytecode = link_code(
#     compiled_output['contracts'][file_name][cont_name],
#     {library_name: lib_contract_address}
# )
# (unlinked,)
my_contract = w3.eth.contract(abi =abi, bytecode=bytecode)
contract_address = deploy_contract(w3,sk, my_contract)
contract_isntance = w3.eth.contract(contract_address, abi=abi)

{'contracts': {'bn254.sol': {'BN254': {'abi': [{'inputs': [{'components': [{'internalType': 'uint256', 'name': 'x', 'type': 'uint256'}, {'internalType': 'uint256', 'name': 'y', 'type': 'uint256'}], 'internalType': 'struct BN254Point', 'name': 'point1', 'type': 'tuple'}, {'components': [{'internalType': 'uint256', 'name': 'x', 'type': 'uint256'}, {'internalType': 'uint256', 'name': 'y', 'type': 'uint256'}], 'internalType': 'struct BN254Point', 'name': 'point2', 'type': 'tuple'}], 'name': 'add', 'outputs': [{'components': [{'internalType': 'uint256', 'name': 'x', 'type': 'uint256'}, {'internalType': 'uint256', 'name': 'y', 'type': 'uint256'}], 'internalType': 'struct BN254Point', 'name': 'ret', 'type': 'tuple'}], 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [{'components': [{'internalType': 'uint256', 'name': 'x', 'type': 'uint256'}, {'internalType': 'uint256', 'name': 'y', 'type': 'uint256'}], 'internalType': 'struct BN254Point', 'name': 'point1', 'type': 'tuple'}, {

In [259]:
def ptsol(point_str):
    return (int(zkbp.from_str(point_str["point"]).x()), int(zkbp.from_str(point_str["point"]).y()))

def ptsol_arr(point_str_arr):
    return [(int(zkbp.from_str(point_str["point"]).x()), int(zkbp.from_str(point_str["point"]).y())) for point_str in point_str_arr]

def ssol(scalar_str):
    return int((scalar_str["scalar"]),16)

range_proof_sol = ( ( ptsol(range_proof_json["A"]), ptsol(range_proof_json["S"]), ptsol(range_proof_json["T1"]), ptsol(range_proof_json["T2"]), ssol(range_proof_json["tau_x"]), ssol(range_proof_json["miu"]), ssol(range_proof_json["tx"]), int(range_proof_json["inner_product_proof"]["a_tag"], 16), int(range_proof_json["inner_product_proof"]["b_tag"],16), ptsol({"point": gh.g}), ptsol({"point": gh.h}), ptsol({"point": cm.get}), ptsol_arr(range_proof_json["inner_product_proof"]["L"]), ptsol_arr(range_proof_json["inner_product_proof"]["R"]), ptsol_arr(ghvec[0]), ptsol_arr(ghvec[1]) ) )

sold_ret = contract_isntance.functions.verify_range_proof(range_proof_sol).call()

print("scalar_cmp:",scalar_cmp)
# print("gycmp:", gy_cmp)
print("ret",sold_ret[0])
print("ret",sold_ret[1])
# print("ret",ret[1] % order)
print("inner_cmp", inner_cmp)
print("point_cmp:", point_cmp)
# print("scalar cmp:", z_cmp)
# print(ret[-1])

print("Ga:", ptsol(ret[-1][-3]))
print("Hb:", ptsol(ret[-1][-2]))
print("ux:", ptsol(ret[-1][-1]))



scalar_cmp: 7287348777628109648367342680462103743216807007752920179971580299437496845241
ret 1
ret 1
inner_cmp (4092044273462250100196902779019191890337916830103873700894858447423788306484, 18219337238173861172345923369345231349068834441561788586943169839348480766788)
point_cmp: (1184411137143888743318766408306828255906377777437964968653005644775923394124, 6672017567699566896835344715922794787275541530814460771253449260756050947229)
Ga: (21657892859384454065700387573477656755105865626214648103969798924344428418948, 16668786466452382622692951371676840976854700177121723595598986240142944613905)
Hb: (19962158596819396949299961524142941783120884084912098593474294699350414056888, 7107041088247748745713043306228826881930910442802152800806896909310601555352)
ux: (21470003018742339193880558059922921130227697180949042415227480289644621053081, 155867670323479615600442538741089593681001523073486764457444742428895478043)


In [260]:
import time
startime =round(time.time() * 1000)
tx = contract_isntance.functions.verify_range_proof(range_proof_sol).buildTransaction({
    'chainId': chain_id,  # Mainnet chain ID
    'gas': 6482780,
    'gasPrice': w3.toWei('20', 'gwei'),
    'nonce': w3.eth.getTransactionCount(eth_account.address),
})

signed_tx = w3.eth.account.sign_transaction(tx, sk)

# Send the transaction
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("time: ",round(time.time() * 1000)-startime, " ms")


print(tx_hash)
print("Transaction hash:", w3.toHex(tx_hash))
print(tx_receipt)

print(tx_receipt["status"])


gas_estimate = contract_isntance.functions.verify_range_proof(range_proof_sol).estimateGas({
    'from': eth_account.address
})
print(f"Estimated Gas: {gas_estimate}")


gas_estimate = contract_isntance.functions.emptytest((int(cm.x()),int(cm.y())),1).estimateGas({
    'from': eth_account.address
})
print(f"Estimated Gas (control): {gas_estimate}")
gas_estimate = contract_isntance.functions.emptytest2((int(cm.x()),int(cm.y())),(int(cm.x()),int(cm.y()))).estimateGas({
    'from': eth_account.address
})
print(f"Estimated Gas (control2 [changed 1 item from uint256 to 2*uint256]): {gas_estimate}")

gas_estimate = contract_isntance.functions.multest((int(cm.x()),int(cm.y())),1).estimateGas({
    'from': eth_account.address
})
print(f"Estimated Gas (mul): {gas_estimate}")
gas_estimate = contract_isntance.functions.multest2((int(cm.x()),int(cm.y())),1).estimateGas({
    'from': eth_account.address
})
print(f"Estimated Gas (mul 2): {gas_estimate}")

gas_estimate = contract_isntance.functions.addtest((int(cm.x()),int(cm.y())),(int(cm.x()),int(cm.y()))).estimateGas({
    'from': eth_account.address
})
print(f"Estimated Gas (add): {gas_estimate}")
gas_estimate = contract_isntance.functions.addtest2((int(cm.x()),int(cm.y())),(int(cm.x()),int(cm.y()))).estimateGas({
    'from': eth_account.address
})
print(f"Estimated Gas (add twice): {gas_estimate}")
#About 5000 in difference between ad and mul

#one add is about 2k and one mul is about 7k

#6.5m total.... (optmized to 5.4m)
# 100k gas for inputs
# LHS == RHS is 200k
# 1.8m computing h * y,inv,,  (300k is for mult, 1.5 is for inv)
# 1m is for computing p 
#ipa is 3m

time:  118  ms
b'\xd8\rs-x\x83\xca\xebT\xd6Y\xdc\xcc\xa3]8\x98q\xc4_\x914rt8\xcc$R<d\xaaP'
Transaction hash: 0xd80d732d7883caeb54d659dccca35d389871c45f9134727438cc24523c64aa50
AttributeDict({'blockHash': HexBytes('0xb54ccc25aa7c55db3febaa4f3d3c6c15d637adbb034c8d2e9cc0d90ab938110c'), 'blockNumber': 156, 'contractAddress': None, 'cumulativeGasUsed': 4057070, 'effectiveGasPrice': '0x4a817c800', 'from': '0xFE3B557E8Fb62b89F4916B721be55cEb828dBd73', 'gasUsed': 4057070, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [261]:
# print(contract_isntance.functions.get().call())
# # print(contract_isntance.functions.set((int(cm.x(),16),int(cm.y(),16)),(int(cm.x(),16),int(cm.y(),16))).call())
# print(cm.x(), ",", cm.y())
# print("mul 1:",contract_isntance.functions.multest((int(cm.x()),int(cm.y())),1).call())
# cm_plus_cm = contract_isntance.functions.addtest((int(cm.x()),int(cm.y())),(int(cm.x()),int(cm.y()))).call()
# print("add:", cm_plus_cm)
# cm_2 = contract_isntance.functions.multest((int(cm.x()),int(cm.y())),2).call()
# print("mul",cm_2)
# assert cm_plus_cm == cm_2

# cm_2 = contract_isntance.functions.multest((int(cm.x()),int(cm.y())),16798108731015832284940804142231733909759579603404752749028378864165570215949).call()
# print("point**order (website):",cm_2)

# cm_2 = contract_isntance.functions.multest((int(cm.x()),int(cm.y())),21888242871839275222246405745257275088548364400416034343698204186575808495617).call()
# print("point**order (Ark):",cm_2)


# cm_2 = contract_isntance.functions.multest((int(cm.x()),int(cm.y())),21888242871839275222246405745257275088548364400416034343698204186575808495618).call()
# print("point**order (Ark):",cm_2)

# cm_2 = contract_isntance.functions.multest((int(cm.x()),int(cm.y())),1).call()
# print("point**1 (Ark):",cm_2)



# # contract_isntance.functions.set(_num=123).transact({"from":eth_account.address})
# print(cm_cm.x())
# print(cm_cm.y())

# # x = 16396164833596690577622818647520533091548481501874234137399799572124395157254
# # y = 15068924686477732314717332919991335554216256854817921729147540176752486115650
# # print(contract_isntance.functions.set((x,y),(x,y)).call())

# print(contract_isntance.functions.get().call())



# print(ptsol(range_proof_json["T1"]))
# print(ssol(range_proof_json["tau_x"]))
# print(ptsol({"point": cm.get}))
# print(range_proof)
# print(ptsol_arr(range_proof_json["inner_product_proof"]["L"]))
# print( ptsol_arr(range_proof_json["inner_product_proof"]["R"]) )
# print(ptsol_arr(ghvec[0]))

# def y1n1(step, count=32, base=1):
#     base_arr = [base]
#     fold = base
#     for i in range(count-1):
#         base_arr.append(base_arr[-1]*step)
#         fold = (fold+ base_arr[-1])%order
#     return fold



# # y1n1(y_cmp)

# arr = [17147861581384430011, 12607530610870547214, 16216658780163595036, 255161181219521184]
# sum = 0
# # arr.reverse()
# print("hash_res_",[int(x) for x in bytes.fromhex(hash_res)] )

# for index, item in enumerate(arr):
#     res =(item).to_bytes(8, "big").hex()
#     print(res)
#     print([int(x) for x in bytes.fromhex(res)])
#     sum +=  item * 2**(64*index)
# print(sum % order)

In [262]:
# print(bin(162))
# print(bin(34))

# print(bin(175))
# print(bin(47))

# print(bin(165))
# print(bin(37))

# print(bin(174))
# print(bin(46))

# print(bin(156))
# print(bin(28))

In [263]:
import hashlib
# def int_to_256bit_byte_array(n):
#     if n<0 or n>=2**256:
#         raise ValueError("123")
    
#     byte_array = n.to_bytes(32,byteorder="little")
#     return byte_array

# # def sha256_chain_point_result_scalar(point_arr):
# #     m = hashlib.sha256()
# #     prefix= (4).to_bytes(1,"big")
# #     a_bytes = b''
# #     for point in point_arr:
# #         a_bytes = a_bytes + prefix+int_to_256bit_byte_array(point[0])+int_to_256bit_byte_array(point[1])
# #     m.update(a_bytes+(b'\0'*4))
# #     return m.hexdigest()

# def sha256_chain_point_result_scalar(point_arr):
#     m = hashlib.sha256()
#     # prefix= (4).to_bytes(1,"big")
#     a_bytes = b''
#     print_q = True
#     for point in point_arr:
#         a_bytes = a_bytes +int_to_256bit_byte_array(point[0])+int_to_256bit_byte_array(point[1])
#         # if print_q:
#         #     # print("xy:",bin(int(a_bytes.hex(),16)))
#         #     x = int_to_256bit_byte_array(point[0])
#         #     y = int_to_256bit_byte_array(point[1])
#         #     # print("x:",[int(item ) for item in x])
#         #     print("y:",[int(item ) for item in y])
#         #     # print_q = False
#     m.update(a_bytes+(b'\0'*4))
#     return m.hexdigest()

# # print(A.to_bytes())
# print("rust implementation:",[bin(byte) for byte in A_bytes])
# # print(bytes(A_bytes).hex())
# print(bin(int(bytes(A_bytes).hex(),16)))

# hash_res =sha256_chain_point_result_scalar([A,S])
# print("hash_Res",bytes.fromhex(hash_res))
# print("hash_res_",[int(x) for x in bytes.fromhex(hash_res)] )
# hash_barr = [int(x) for x in bytes.fromhex(hash_res)] 
# sum = 0
# sum_big =0
# sum_mod = 0
# sum_bigmod = 0
# for index,power in enumerate(hash_barr):
#     sum += 2**(8*index) * power
#     sum_big += 2**(8*(len(hash_barr)-1-index)) * power
# print(len(hash_barr))
# print("hash byte little endian sum:", sum % order )
# print("hash byte big endian sum:", sum_big % order )
# sum =0 
# sum_big =0
# for i in range(4):
#     hash_barr_prime = hash_barr[i*8:(i+1)*8]
#     print(len(hash_barr_prime), "i*8:", i*8, "\t i+1*8", (i+1)*8)
#     for index,power in enumerate(hash_barr_prime):
#         sum_mod += 2**(8*index) * power
#         sum_bigmod += 2**(8*(len(hash_barr)-1-index)) * power
#     # sum += sum_mod * 2**(64*i)
#     # sum_big += sum_bigmod * 2**(64*i)
#     sum += sum_mod * 2**(64*(3-i))
#     sum_big += sum_bigmod * 2**(64*(3-i))

# print("(little) hash byte little endian sum:", sum % order )
# print("(little) hash byte big endian sum:", sum_big % order )

# print ("Parse to int (no red)",int(bytes.fromhex(hash_res).hex(),16) )
# print ("Parse to int",int(bytes.fromhex(hash_res).hex(),16) % order)

# # for i in range(4):

# print("CMP: from_bytes()",cm.from_bytes(bytes.fromhex(hash_res)), " ==? ", y_cmp) #this one is correct.
# # print(y_cmp% order)
# print ("int:",bin(int(bytes.fromhex(hash_res).hex(),16) % order))
# print("ycp:",bin(y_cmp))
# with pipes() as (out,err):
#     print(cm.from_bytes(bytes.fromhex(hash_res)) == str(y_cmp))
# # print(out.read())
# # rev = bytearray.fromhex(hash_res)
# # rev.reverse()
# # print("from_bytes()",cm.from_bytes(rev))

# print((-2) % 21888242871839275222246405745257275088696311157297823662689037894645226208583)

In [264]:
#   function modExp(uint256 base, uint256 exponent, uint256 modulus) internal view returns (uint256) {
#         uint256[6] memory input;
#         uint256[1] memory output;
#         input[0] = 0x20;  // length_of_BASE
#         input[1] = 0x20;  // length_of_EXPONENT
#         input[2] = 0x20;  // length_of_MODULUS
#         input[3] = base;
#         input[4] = exponent;
#         input[5] = modulus;
#         assembly {
#             if iszero(staticcall(not(0), 5, input, 0xc0, output, 0x20)) {
#                 revert(0, 0)
#             }
#         }
#         return output[0];
#     }